In [109]:
import itertools
import torch
import torch.optim as optim
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import random
from numpy.linalg import norm
import numpy as np
import math
import time
from torch.utils.data import DataLoader, TensorDataset
from sklearn.decomposition import PCA

In [110]:
import sys
root = '../../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [111]:
def CreateDataset(root, num_classes, batch_size, train = 1, HPO = False):
    if HPO == False:
        if train == 1:
            #Create X_train_tensor
            X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)

            #pca = PCA(n_components=128)
            #pca.fit(X_train)
            #X_train = torch.tensor(pca.transform(X_train), dtype=torch.float)
            #X_test = torch.tensor(pca.transform(X_test), dtype=torch.float)
            X_train_tensor = torch.tensor(X_train)

            #Create Y_train_tensor
            y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)
            y_train_tensor = torch.tensor(y_train, dtype=torch.long)
            y_train_tensor = torch.nn.functional.one_hot(y_train_tensor, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)

            #Create indices
            indices_train = torch.arange(len(X_train_tensor))

            dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
            train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return train_loader

        elif train == 2:
            X_validation = np.load( root + "Features/val_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
            X_validation_tensor = torch.tensor(X_validation)

            y_validation = np.load( root + "Features/val_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
            y_validation_tensor = torch.tensor(y_validation, dtype=torch.long)
            y_validation_tensor = torch.nn.functional.one_hot(y_validation_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

            #Create indices
            indices_validation = torch.arange(len(X_validation_tensor))

            dataset = TensorDataset(X_validation_tensor, y_validation_tensor, indices_validation)
            validation_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return validation_loader

        elif train == 3:
            X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
            X_test_tensor = torch.tensor(X_test)

            y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
            y_test_tensor = torch.tensor(y_test, dtype=torch.long)
            y_test_tensor = torch.nn.functional.one_hot(y_test_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

            #Create indices
            indices_test = torch.arange(len(X_test_tensor))

            dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
            test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return test_loader



    if HPO == True:
        if train == 1:
            #Create X_train_tensor
            X_train = np.load( root + "Features/X_hpo_Nus.npy" ) # Shape = (45000, 4096)
            X_train_tensor = torch.tensor(X_train)
            print(X_train_tensor.shape)

            #Create Y_train_tensor
            y_train = np.load( root + "Features/y_hpo_Nus.npy" ) # Shape = (45000,)
            y_train_tensor = torch.tensor(y_train, dtype=torch.long)
            #y_train_tensor = torch.nn.functional.one_hot(y_train_tensor)#, num_classes) #One-Hot Encoded -> Shape = (45000, num_classes)
            print(y_train_tensor.shape)

            #Create indices
            indices_train = torch.arange(len(X_train_tensor))

            dataset = TensorDataset(X_train_tensor, y_train_tensor, indices_train)
            train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return train_loader

        elif train == 2:
            X_validation = np.load( root + "Features/X_val_Nus.npy" ) # Shape = (10000, 4096)
            X_validation_tensor = torch.tensor(X_validation)
            print(X_validation_tensor.shape)

            y_validation = np.load( root + "Features/y_val_Nus.npy" ) # Shape = (10000,)
            y_validation_tensor = torch.tensor(y_validation, dtype=torch.long)
            #y_validation_tensor = torch.nn.functional.one_hot(y_validation_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)
            print(y_validation_tensor.shape)
            print(y_validation_tensor)

            #Create indices
            indices_validation = torch.arange(len(X_validation_tensor))

            dataset = TensorDataset(X_validation_tensor, y_validation_tensor, indices_validation)
            validation_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return validation_loader

        elif train == 3:
            X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy") # Shape = (10000, 4096)
            X_test_tensor = torch.tensor(X_test)

            y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy") # Shape = (10000,)
            y_test_tensor = torch.tensor(y_test, dtype=torch.long)
            y_test_tensor = torch.nn.functional.one_hot(y_test_tensor, num_classes) #One-Hot Encoded -> Shape = (10000, num_classes)

            #Create indices
            indices_test = torch.arange(len(X_test_tensor))

            dataset = TensorDataset(X_test_tensor, y_test_tensor, indices_test)
            test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            return test_loader

In [112]:
train_loader = CreateDataset(root, num_classes = 21, batch_size = 128, train = 1, HPO=True)
#test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2)
validation_loader = CreateDataset(root, num_classes = 21, batch_size = 128, train = 2, HPO = True)

torch.Size([5002, 4096])
torch.Size([5002, 21])
torch.Size([502, 4096])
torch.Size([502, 21])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [1, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [113]:
class CustomNN(nn.Module):
    def __init__(self, bits):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, bits),    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights based on paper
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases based on paper

    def forward(self, x):
        return self.fc_layers(x)

In [114]:
class DPSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DPSHLoss, self).__init__()
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        s = (y @ self.Y.t() > 0).float().clamp(max = 1)
        inner_product = u @ self.U.t() * 0.5

        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product

        likelihood_loss = likelihood_loss.mean()

        quantization_loss = eta * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

In [115]:
# Define the grid

def DPSH(device: torch.device, train_size: int, n_classes: int, bit: int, num_epoch: int, batch_size: int, eta_values: list, wd_values: list, lr_values: list):

    train_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 1, HPO=True)
    #test_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2)
    validation_loader = CreateDataset(root, num_classes = 10, batch_size = 128, train = 2, HPO = True)




    param_grid = {
        'eta': eta_values,
        'learning_rate': lr_values,
        #'batch_size': [16, 32, 64],
        'weight_decay': wd_values
    }

    customLoss = DPSHLoss(train_size, n_classes, bit)


    # Get all combinations of parameters
    keys, values = zip(*param_grid.items())
    parameter_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Evaluate each parameter combination
    best_params = None
    best_map = 0

    for params in parameter_combinations:
        print(f"Testing combination: {params}")
    
        # Initialize loss function with specific parameters
        #loss_fn = customLoss()

        # Initialize model and optimizer
        model = CustomNN(bits = bit).to(device)
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay = params['weight_decay'])

        # Train the model
        for epoch in range(num_epoch):  # Example epoch count
            current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))
            print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            "DPSH", epoch + 1, num_epoch, current_time, bit, "CIFAR"), end="")
            model.train()
            train_loss = 0
            for image, label, ind in train_loader:
                image = image.to(device)
                label = label.to(device)

                optimizer.zero_grad()
                u = model(image)

                loss = customLoss(u, label.float(), ind, eta = params['eta'])
                train_loss += loss.item()

                loss.backward()
                optimizer.step()

            train_loss = train_loss / (train_size / batch_size)
            print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))
        # Validate the model
        model.eval()
        hash_train = []
        label_train = []

        hash_val = []
        label_val = []

        with torch.no_grad():
            for image, label, ind in train_loader:
                image = image.to(device)
                label_batch = label.to(device)
                hash_train_batch = (model(image)).sign()
                hash_train.append(hash_train_batch)
                label_train.append(label_batch)
                #hash_codes_matrix = np.vstack(hash_code_batches)
                #print(hash_codes_matrix.shape) 

            hash_train = torch.cat(hash_train, dim = 0).cpu().numpy()
            label_train = torch.cat(label_train, dim = 0).cpu().numpy()

            
            for image, label, ind in validation_loader:
                image = image.to(device)
                label_batch = label.to(device)
                hash_val_batch = (model(image)).sign()
                hash_val.append(hash_val_batch)
                label_val.append(label_batch)
            hash_val = torch.cat(hash_val, dim = 0).cpu().numpy()
            label_val = torch.cat(label_val, dim = 0).cpu().numpy()

            map = meanAveragePrecision(training_hashes = hash_train, training_labels = label_train, test_hashes = hash_val, test_labels = label_val)

            print(f"Validation mAP: {map:.4f}")

            # Update best parameters
            if map > best_map:
                best_map = map
                best_params = params

    print("Best Parameters:", best_params)
    print("Best mAP:", best_map)
    return best_params

In [116]:
DPSH12Nus = DPSH(device, 5002, 21, 12, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
DPSH24Nus = DPSH(device, 5002, 21, 24, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
DPSH32Nus = DPSH(device, 5002, 21, 32, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])
DPSH48Nus = DPSH(device, 5002, 21, 48, 150, 128, [0.01, 0.05, 0.1], [1e-6, 1e-5, 1e-4], [1e-6, 1e-5, 1e-4])

torch.Size([5002, 4096])
torch.Size([5002, 21])
torch.Size([502, 4096])
torch.Size([502, 21])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [1, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
Testing combination: {'eta': 0.01, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH[ 1/150][12:37:29] bit:12, dataset:CIFAR, training... loss:0.72467
DPSH[ 2/150][12:37:30] bit:12, dataset:CIFAR, training... loss:0.71062
DPSH[ 3/150][12:37:30] bit:12, dataset:CIFAR, training... loss:0.70690
DPSH[ 4/150][12:37:30] bit:12, dataset:CIFAR, training... loss:0.69193
DPSH[ 5/150][12:37:30] bit:12, dataset:CIFAR, training... loss:0.67190
DPSH[ 6/150][12:37:30] bit:12, dataset:CIFAR, training... loss:0.64968
DPSH[ 7/150][12:37:31] bit:12, dataset:CIFAR, training... loss:0.62716
DPSH[ 8/150][12:37:31] bit:12, dataset:CIFAR, training... loss:0.61079
DPSH[ 9/150][12:37:31] bit:12, dataset

100%|██████████| 502/502 [00:01<00:00, 427.51it/s]


Validation mAP: 0.7292
Testing combination: {'eta': 0.01, 'learning_rate': 1e-06, 'weight_decay': 1e-05}
DPSH[ 1/150][12:38:04] bit:12, dataset:CIFAR, training... loss:0.73613
DPSH[ 2/150][12:38:04] bit:12, dataset:CIFAR, training... loss:0.71677
DPSH[ 3/150][12:38:04] bit:12, dataset:CIFAR, training... loss:0.71401
DPSH[ 4/150][12:38:04] bit:12, dataset:CIFAR, training... loss:0.70963
DPSH[ 5/150][12:38:05] bit:12, dataset:CIFAR, training... loss:0.70450
DPSH[ 6/150][12:38:05] bit:12, dataset:CIFAR, training... loss:0.69626
DPSH[ 7/150][12:38:05] bit:12, dataset:CIFAR, training... loss:0.68379
DPSH[ 8/150][12:38:05] bit:12, dataset:CIFAR, training... loss:0.66582
DPSH[ 9/150][12:38:05] bit:12, dataset:CIFAR, training... loss:0.64670
DPSH[10/150][12:38:06] bit:12, dataset:CIFAR, training... loss:0.62826
DPSH[11/150][12:38:06] bit:12, dataset:CIFAR, training... loss:0.61381
DPSH[12/150][12:38:06] bit:12, dataset:CIFAR, training... loss:0.60067
DPSH[13/150][12:38:06] bit:12, dataset:CIFA

100%|██████████| 502/502 [00:01<00:00, 426.00it/s]


Validation mAP: 0.7255
Testing combination: {'eta': 0.01, 'learning_rate': 1e-06, 'weight_decay': 0.0001}
DPSH[ 1/150][12:39:14] bit:12, dataset:CIFAR, training... loss:0.72155
DPSH[ 2/150][12:39:15] bit:12, dataset:CIFAR, training... loss:0.71468
DPSH[ 3/150][12:39:16] bit:12, dataset:CIFAR, training... loss:0.71169
DPSH[ 4/150][12:39:16] bit:12, dataset:CIFAR, training... loss:0.70603
DPSH[ 5/150][12:39:17] bit:12, dataset:CIFAR, training... loss:0.69930
DPSH[ 6/150][12:39:17] bit:12, dataset:CIFAR, training... loss:0.68758
DPSH[ 7/150][12:39:18] bit:12, dataset:CIFAR, training... loss:0.67238
DPSH[ 8/150][12:39:18] bit:12, dataset:CIFAR, training... loss:0.65370
DPSH[ 9/150][12:39:19] bit:12, dataset:CIFAR, training... loss:0.63444
DPSH[10/150][12:39:19] bit:12, dataset:CIFAR, training... loss:0.61778
DPSH[11/150][12:39:20] bit:12, dataset:CIFAR, training... loss:0.60610
DPSH[12/150][12:39:21] bit:12, dataset:CIFAR, training... loss:0.59536
DPSH[13/150][12:39:21] bit:12, dataset:CIF

...

Validation mAP: 0.8130
Testing combination: {'eta': 0.1, 'learning_rate': 0.0001, 'weight_decay': 1e-05}
DPSH[ 1/150][14:04:44] bit:48, dataset:CIFAR, training... loss:0.65134
DPSH[ 2/150][14:04:44] bit:48, dataset:CIFAR, training... loss:0.61887
DPSH[ 3/150][14:04:45] bit:48, dataset:CIFAR, training... loss:0.57771
DPSH[ 4/150][14:04:45] bit:48, dataset:CIFAR, training... loss:0.55237
DPSH[ 5/150][14:04:45] bit:48, dataset:CIFAR, training... loss:0.53344
DPSH[ 6/150][14:04:45] bit:48, dataset:CIFAR, training... loss:0.51464
DPSH[ 7/150][14:04:45] bit:48, dataset:CIFAR, training... loss:0.50270
DPSH[ 8/150][14:04:45] bit:48, dataset:CIFAR, training... loss:0.48739
DPSH[ 9/150][14:04:46] bit:48, dataset:CIFAR, training... loss:0.47132
DPSH[10/150][14:04:46] bit:48, dataset:CIFAR, training... loss:0.45783
DPSH[11/150][14:04:46] bit:48, dataset:CIFAR, training... loss:0.44305
DPSH[12/150][14:04:46] bit:48, dataset:CIFAR, training... loss:0.42696
DPSH[13/150][14:04:46] bit:48, dataset:CIFA

100%|██████████| 502/502 [00:01<00:00, 426.66it/s]


Validation mAP: 0.8210
Testing combination: {'eta': 0.1, 'learning_rate': 0.0001, 'weight_decay': 0.0001}
DPSH[ 1/150][14:05:12] bit:48, dataset:CIFAR, training... loss:0.64844
DPSH[ 2/150][14:05:12] bit:48, dataset:CIFAR, training... loss:0.62054
DPSH[ 3/150][14:05:13] bit:48, dataset:CIFAR, training... loss:0.57801
DPSH[ 4/150][14:05:13] bit:48, dataset:CIFAR, training... loss:0.55537
DPSH[ 5/150][14:05:13] bit:48, dataset:CIFAR, training... loss:0.53449
DPSH[ 6/150][14:05:13] bit:48, dataset:CIFAR, training... loss:0.51529
DPSH[ 7/150][14:05:13] bit:48, dataset:CIFAR, training... loss:0.50152
DPSH[ 8/150][14:05:13] bit:48, dataset:CIFAR, training... loss:0.49196
DPSH[ 9/150][14:05:14] bit:48, dataset:CIFAR, training... loss:0.47775
DPSH[10/150][14:05:14] bit:48, dataset:CIFAR, training... loss:0.46390
DPSH[11/150][14:05:14] bit:48, dataset:CIFAR, training... loss:0.45077
DPSH[12/150][14:05:14] bit:48, dataset:CIFAR, training... loss:0.44065
DPSH[13/150][14:05:14] bit:48, dataset:CIF

100%|██████████| 502/502 [00:01<00:00, 431.17it/s]

Validation mAP: 0.8160
Best Parameters: {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 1e-06}
Best mAP: 0.8243455029800065


In [ ]:
DPSH48Cifar = {'eta': 0.05, 'learning_rate': 1e-05, 'weight_decay': 1e-05}
DPSH32Cifar = {'eta': 0.1, 'learning_rate': 1e-05, 'weight_decay': 1e-05}
DPSH24Cifar = {'eta': 0.1, 'learning_rate': 1e-05, 'weight_decay': 0.0001}
DPSH12Cifar = {'eta': 0.01, 'learning_rate': 1e-05, 'weight_decay': 1e-06}
DPSH12Img = {'eta': 0.1, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH24Img = {'eta': 0.1, 'learning_rate': 1e-06, 'weight_decay': 1e-05}
DPSH32Img = {'eta': 0.1, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH48Img = {'eta': 0.1, 'learning_rate': 1e-06, 'weight_decay': 1e-06}
DPSH12Nus = {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 1e-05}
DPSH24Nus = {'eta': 0.01, 'learning_rate': 0.0001, 'weight_decay': 1e-05}
DPSH32Nus = {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 0.0001}
DPSH48Nus = {'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 1e-06}

{'eta': 0.05, 'learning_rate': 0.0001, 'weight_decay': 1e-06}

: 